## Estudio de componentes principales con LDA

##### En el presente documento se utilizará el módulo LDA (Latent Dirichlet Allocation) para encontrar distintos tópicos a través de los dos datasets: el compuesto por tuits y el compuesto por menúes de comidas.

In [45]:
import nltk
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from gensim.corpora import Dictionary

### Para el corpus formado por tuits extraídos

In [36]:
dataset = pd.read_csv('preprocessed_tweets_ww.csv', index_col=False)
dataset.dropna(inplace=True)

In [37]:
dataset

,screen_names,texts,dates,langs
0,excessvs,vida italiana llena cliches mismo mano rozo kh...,2021-04-28 00:20:33,es
1,EuSiso,meses comer hamburguesa pasta cuerpo bueno chi...,2021-04-28 00:20:28,es
2,_Sosaluis_,ganas hamburguesa,2021-04-28 00:20:14,es
3,SierraTeff,milanesa hamburguesa team cafinapp,2021-04-28 00:20:06,es
4,DraArias,haciendo pan brioche hamburguesa saboreando,2021-04-28 00:19:48,es
...,...,...,...,...
9463,isaprzg,unas ganas asquerosas hamburguesa papas gajo n...,2021-04-25 21:58:50,es
9464,linternasazules,seis tarrinas helado limon galletas chocolate ...,2021-04-25 21:58:44,es
9465,curly_Bae_bae,copio grr usa marcar comes mayonesa dulce lech...,2021-04-25 21:58:44,es
9466,wafibreadbar,usa marcar comes mostaza pepinillo cilantro pi...,2021-04-25 21:58:40,es


In [35]:
LDA = gensim.models.ldamodel.LdaModel

#### Como los tuits fueron recolectados si contenían la palabra `hamburguesa`, la eliminaremos del vocabulario para evitar ruido en la detección de tópicos.

In [32]:
texts = list(dataset['texts'].str.replace("hamburguesa", ""))
tokenized_texts = [text.split() for text in texts]

In [33]:
dictionary = Dictionary(tokenized_texts)
doc_term_matrix = [dictionary.doc2bow(line) for line in tokenized_texts]

In [36]:
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=5, random_state=100,
                chunksize=1000, passes=50,iterations=100)

In [40]:
topics = lda_model.print_topics()
topics

[(0,
  '0.032*"ganas" + 0.022*"comer" + 0.013*"voy" + 0.012*"s" + 0.011*"mas" + 0.010*"si" + 0.009*"buena" + 0.009*"mayonesa" + 0.007*"q" + 0.007*"mc"'),
 (1,
  '0.021*"si" + 0.019*"m" + 0.015*"probe" + 0.015*"twt" + 0.014*"antojo" + 0.011*"nunca" + 0.010*"q" + 0.009*"necesito" + 0.009*"i" + 0.009*"gusta"'),
 (2,
  '0.042*"quiero" + 0.016*"papas" + 0.014*"comer" + 0.011*"cheddar" + 0.010*"ganas" + 0.009*"si" + 0.009*"solo" + 0.009*"queso" + 0.008*"q" + 0.007*"comida"'),
 (3,
  '0.093*"pizza" + 0.088*"mostaza" + 0.088*"sushi" + 0.087*"cilantro" + 0.086*"piña" + 0.085*"pepinillo" + 0.081*"usa" + 0.077*"comes" + 0.076*"brocoli" + 0.068*"marcar"'),
 (4,
  '0.011*"si" + 0.010*"q" + 0.008*"mas" + 0.008*"comiendo" + 0.008*"ver" + 0.007*"hoy" + 0.007*"siempre" + 0.006*"comi" + 0.006*"food" + 0.006*"mejor"')]

##### Si bien no es fácil distinguir tópicos coherentes de los vectores obtenidos, podemos buscar algunos patrones:

#### Análisis del tópico 0

In [41]:
topics[0]

(0,
 '0.032*"ganas" + 0.022*"comer" + 0.013*"voy" + 0.012*"s" + 0.011*"mas" + 0.010*"si" + 0.009*"buena" + 0.009*"mayonesa" + 0.007*"q" + 0.007*"mc"')

##### Por las palabras que componen a este tópico, se podría pensar que el mismo reúne los comentarios que expresan en deseo de comer una hamburguesa. Aunque con poca probabilidad, aparece el token "Mc" (haciendo referencia al restaurante de comida rápida Mc Donald's).

#### Análisis del tópico 3

In [42]:
topics[3]

(3,
 '0.093*"pizza" + 0.088*"mostaza" + 0.088*"sushi" + 0.087*"cilantro" + 0.086*"piña" + 0.085*"pepinillo" + 0.081*"usa" + 0.077*"comes" + 0.076*"brocoli" + 0.068*"marcar"')

La mayoría de los componentes de este vector pertenecen a la cadena mencionada anteriormente formada por cientos de tuits, por lo que no sorprende que el algoritmo haya detectado las palabras que lo componen como un tema.

### Para el corpus formado por menúes de comida

In [58]:
menues_samples = pd.read_csv('datasets/food_preprocessed.csv')
menues_samples['description'].head()

0    hamburguesa vegetariana ciser hamburguesa garb...
1    godiee burger g carne lentejas pan rucula alba...
2    hamburguesa vegetariana doble hamburguesa lent...
3    hamburguesa vegetariana pan artesanal sesamo h...
4    hamburguesa vegetariana pan batta dos hamburgu...
Name: description, dtype: object

In [59]:
menues_LDA = gensim.models.ldamodel.LdaModel

#### Aplicamos el mismo criterio y removemos la ocurrencia del string `hamburguesa`

In [60]:
menues_texts = list(menues_samples['description'].str.replace("hamburguesa", ""))
menues_tokenized_texts = [text.split() for text in menues_texts]

In [62]:
menues_dictionary = Dictionary(menues_tokenized_texts)
menues_doc_term_matrix = [menues_dictionary.doc2bow(doc) for doc in menues_tokenized_texts]

In [68]:
menues_lda_model = LDA(corpus=menues_doc_term_matrix, id2word=menues_dictionary, num_topics=5, random_state=100,
                chunksize=1000, passes=50,iterations=100)

In [69]:
topics = menues_lda_model.print_topics()
topics

[(0,
  '0.068*"cheddar" + 0.047*"cebolla" + 0.045*"doble" + 0.043*"carne" + 0.042*"queso" + 0.039*"panceta" + 0.037*"salsa" + 0.035*"pan" + 0.030*"burger" + 0.023*"g"'),
 (1,
  '0.071*"promo" + 0.057*"ml" + 0.044*"pollo" + 0.037*"coca" + 0.036*"cola" + 0.028*"combo" + 0.025*"olimpico" + 0.024*"burger" + 0.023*"refresco" + 0.020*"bebida"'),
 (2,
  '0.098*"fritas" + 0.090*"papas" + 0.083*"tomate" + 0.079*"lechuga" + 0.069*"huevo" + 0.054*"mayonesa" + 0.051*"jamon" + 0.039*"muzzarella" + 0.036*"pan" + 0.031*"completa"'),
 (3,
  '0.228*"sandwich" + 0.113*"caliente" + 0.089*"jamon" + 0.080*"muzzarella" + 0.079*"queso" + 0.034*"comun" + 0.028*"tomate" + 0.023*"unidades" + 0.019*"napolitano" + 0.018*"pan"'),
 (4,
  '0.066*"plato" + 0.054*"casera" + 0.050*"guarnicion" + 0.042*"menu" + 0.039*"gustos" + 0.034*"zanahoria" + 0.031*"viernes" + 0.030*"carne" + 0.030*"ensalada" + 0.030*"pan"')]

##### A diferencia del dataset anterior, es más fácil distinguir tópicas en con estos vectores

#### Análisis del tópico 0

In [70]:
topics[0]

(0,
 '0.068*"cheddar" + 0.047*"cebolla" + 0.045*"doble" + 0.043*"carne" + 0.042*"queso" + 0.039*"panceta" + 0.037*"salsa" + 0.035*"pan" + 0.030*"burger" + 0.023*"g"')

Por las palabras que lo componen `carne`, `burguer`, `pan`, `cheddar`, `queso`, `cebolla` parece claro indicar que está concentrando la información de los menúes que ofrecen hamburguesas al pan.

#### Análisis del tópico 1

In [71]:
topics[1]

(1,
 '0.071*"promo" + 0.057*"ml" + 0.044*"pollo" + 0.037*"coca" + 0.036*"cola" + 0.028*"combo" + 0.025*"olimpico" + 0.024*"burger" + 0.023*"refresco" + 0.020*"bebida"')

Si bien hay algunos términos mezclados, no es difícil inferir que este tópico representa la existencia de las "promos" (o "combos") siendo combinaciones entre bebidas (representadas por las palabras `bebida`, `refresco`, `coca`, `cola` y `ml`) y comidas (`pollo`, `olimpico`, `burguer`).

##### Como conclusión, vale destacar cómo los tópicos se hacen más claros utilizando el dataset compuesto por menúes, donde no hay tantas variedades como el corpus compuesto por tuits.